In [1]:
import pandas as pd
from io import StringIO
import requests
import re
import matplotlib.pyplot as plt
import warnings

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
warnings.filterwarnings("ignore")

In [2]:
# Function to list files in a GitHub repository
def list_github_files(repo_name: str) -> list | None:
    url = f"https://api.github.com/repos/{repo_name}/contents/"
    response = requests.get(url)
    if response.status_code == 200:
        files = response.json()
        file_names = [file["name"] for file in files]
        return file_names
    else:
        print(f"Failed to retrieve files: {response.status_code}")


def read_csv_from_github(repo_name: str, file_name: str) -> pd.DataFrame:
    """
    Reads a CSV file from a GitHub repository into a Pandas DataFrame.
    Parameters:
        repo_name (str): The name of the GitHub repository (e.g., "owner/repo").
        file_name (str): The name of the CSV file.
    Returns:
        pd.DataFrame: A Pandas DataFrame containing the data from the CSV file.
    """

    file_url = f"https://raw.githubusercontent.com/{repo_name}/master/{file_name}"
    response = requests.get(file_url, verify=False)
    csv_content = response.content.decode("utf-8")
    df = pd.read_csv(StringIO(csv_content))
    return df

In [3]:
repo_name = "JeffSackmann/tennis_atp"
file_names = list_github_files(repo_name)

# Filter files to only include CSV files
file_names = [file for file in file_names if file.endswith(".csv")]

# Filter files to only include those that follow the "atp_matches_YYYY" pattern
atp_matches = [file for file in file_names if re.match(r"atp_matches_\d{4}\.csv", file)]

# Read and aggregate ATP matches data
atp_matches_dfs = [read_csv_from_github(repo_name, file) for file in atp_matches]
atp_matches_df = pd.concat(atp_matches_dfs, ignore_index=True)

# Display df info for ATP matches
print(
    "ATP Matches DataFrame Info:",
    "\nShape:",
    atp_matches_df.shape,
    "\nColumns:",
    atp_matches_df.columns,
)

ATP Matches DataFrame Info: 
Shape: (194996, 49) 
Columns: Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')


In [4]:
# Convert the 'tourney_date' column to datetime format
atp_matches_df["tourney_date"] = pd.to_datetime(
    atp_matches_df["tourney_date"], format="%Y%m%d"
)
# Create tournay_year column
atp_matches_df["tourney_year"] = atp_matches_df["tourney_date"].dt.year
# Filter for matches after selected year
selected_year = 1991
atp_matches_df = atp_matches_df[
    atp_matches_df["tourney_year"] >= selected_year
].reset_index(drop=True)
# Filter for tourney levels in ['G', 'F', 'M', 'A']
atp_matches_df = atp_matches_df[
    atp_matches_df["tourney_level"].isin(["G", "F", "M", "A"])
].reset_index(drop=True)
# Exclude Laver Cup matches
atp_matches_df = atp_matches_df[
    ~atp_matches_df["tourney_name"].str.contains("Laver Cup")
].reset_index(drop=True)

# Count the number of NaN values for each column in the ATP matches DataFrame
nan_counts_matches = atp_matches_df.isna().sum() / atp_matches_df.shape[0] * 100
print("Dataframe shape: ", atp_matches_df.shape)
print("NaN % in ATP matches DataFrame from {}:".format(selected_year))
print(nan_counts_matches)

Dataframe shape:  (98355, 50)
NaN % in ATP matches DataFrame from 1991:
tourney_id             0.000000
tourney_name           0.000000
surface                0.000000
draw_size              0.000000
tourney_level          0.000000
tourney_date           0.000000
match_num              0.000000
winner_id              0.000000
winner_seed           55.391185
winner_entry          86.417569
winner_name            0.000000
winner_hand            0.000000
winner_ht              0.095572
winner_ioc             0.000000
winner_age             0.000000
loser_id               0.000000
loser_seed            75.417620
loser_entry           77.741853
loser_name             0.000000
loser_hand             0.000000
loser_ht               0.721875
loser_ioc              0.000000
loser_age              0.012201
score                  0.000000
best_of                0.000000
round                  0.000000
minutes                4.663718
w_ace                  1.810787
w_df                   1.810787


In [5]:
atp_matches_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_year
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1,101142,1.0,NaN,Emilio Sanchez,R,180.0,ESP,25.6,101746,NaN,NaN,Renzo Furlan,R,175.0,ITA,20.6,6-4 6-1,3,R32,63.0,1.0,0.0,53.0,37.0,30.0,7.0,9.0,5.0,6.0,3.0,0.0,46.0,30.0,17.0,7.0,8.0,2.0,6.0,9.0,1487.0,78.0,459.0,1991
1,1991-301,Auckland,Hard,32.0,A,1991-01-07,2,101613,NaN,Q,Malivai Washington,R,180.0,USA,21.5,100587,NaN,WC,Steve Guy,R,188.0,NZL,31.8,6-3 6-2,3,R32,72.0,5.0,1.0,56.0,25.0,17.0,20.0,9.0,1.0,2.0,4.0,7.0,56.0,30.0,22.0,6.0,8.0,7.0,11.0,94.0,371.0,220.0,114.0,1991
2,1991-301,Auckland,Hard,32.0,A,1991-01-07,3,101179,NaN,NaN,Jean Philippe Fleurian,R,185.0,FRA,25.3,101601,NaN,WC,Brett Steven,R,185.0,NZL,21.6,2-6 6-1 6-2,3,R32,101.0,2.0,4.0,80.0,55.0,35.0,16.0,12.0,2.0,4.0,1.0,3.0,68.0,43.0,24.0,14.0,11.0,4.0,8.0,77.0,468.0,212.0,116.0,1991
3,1991-301,Auckland,Hard,32.0,A,1991-01-07,4,101117,NaN,NaN,Eric Jelen,R,180.0,GER,25.8,101332,8.0,NaN,Gilad Bloom,L,173.0,ISR,23.8,6-3 1-6 6-4,3,R32,108.0,0.0,1.0,82.0,55.0,35.0,14.0,13.0,6.0,10.0,3.0,2.0,96.0,61.0,38.0,15.0,13.0,8.0,12.0,65.0,502.0,72.0,483.0,1991
4,1991-301,Auckland,Hard,32.0,A,1991-01-07,5,101901,NaN,Q,Chuck Adams,R,185.0,USA,19.7,101735,3.0,NaN,Richard Fromberg,R,196.0,AUS,20.6,6-3 6-4,3,R32,65.0,4.0,4.0,65.0,46.0,34.0,12.0,10.0,2.0,2.0,1.0,3.0,49.0,25.0,21.0,12.0,9.0,4.0,6.0,190.0,142.0,28.0,876.0,1991


## Player Stats Historical

In [10]:
atp_matches_df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'tourney_year'],
      dtype='object')

In [ ]:
w_cols = [
    "tourney_id",
    "tourney_name",
    "surface",
    "draw_size",
    "tourney_level",
    "tourney_date",
    "tourney_year",
    "match_num",  # tournament info
    "winner_id",
    "winner_seed",
    "winner_entry",
    "winner_name",
    "winner_hand",
    "winner_ht",
    "winner_ioc",
    "winner_age",
    "winner_rank",
    "winner_rank_points",  # winner info
    "w_ace",
    "w_df",
    "w_svpt",
    "w_1stIn",
    "w_1stWon",
    "w_2ndWon",
    "w_SvGms",
    "w_bpSaved",
    "w_bpFaced",  # winner stats
    "score",
    "best_of",
    "round",
    "minutes",  # match info
]

l_cols = [
    "tourney_id",
    "tourney_name",
    "surface",
    "draw_size",
    "tourney_level",
    "tourney_date",
    "tourney_year",
    "match_num",  # tournament info
    "loser_id",
    "loser_seed",
    "loser_entry",
    "loser_name",
    "loser_hand",
    "loser_ht",
    "loser_ioc",
    "loser_age",
    "loser_rank",
    "loser_rank_points",  # loser info
    "l_ace",
    "l_df",
    "l_svpt",
    "l_1stIn",
    "l_1stWon",
    "l_2ndWon",
    "l_SvGms",
    "l_bpSaved",
    "l_bpFaced",  # loser stats
    "score",
    "best_of",
    "round",
    "minutes",  # match info
]

In [24]:
player_stats_hist_df = atp_matches_df[w_cols].copy()
player_stats_hist_df_l = atp_matches_df[l_cols].copy()

# add a column to flag winner and loser
player_stats_hist_df["results"] = "W"
player_stats_hist_df_l["results"] = "L"
player_stats_hist_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,score,best_of,round,minutes,results
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,1,101142,1.0,NaN,Emilio Sanchez,R,180.0,ESP,25.6,9.0,1487.0,1.0,0.0,53.0,37.0,30.0,7.0,9.0,5.0,6.0,6-4 6-1,3,R32,63.0,W
1,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,2,101613,NaN,Q,Malivai Washington,R,180.0,USA,21.5,94.0,371.0,5.0,1.0,56.0,25.0,17.0,20.0,9.0,1.0,2.0,6-3 6-2,3,R32,72.0,W
2,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,3,101179,NaN,NaN,Jean Philippe Fleurian,R,185.0,FRA,25.3,77.0,468.0,2.0,4.0,80.0,55.0,35.0,16.0,12.0,2.0,4.0,2-6 6-1 6-2,3,R32,101.0,W
3,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,4,101117,NaN,NaN,Eric Jelen,R,180.0,GER,25.8,65.0,502.0,0.0,1.0,82.0,55.0,35.0,14.0,13.0,6.0,10.0,6-3 1-6 6-4,3,R32,108.0,W
4,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,5,101901,NaN,Q,Chuck Adams,R,185.0,USA,19.7,190.0,142.0,4.0,4.0,65.0,46.0,34.0,12.0,10.0,2.0,2.0,6-3 6-4,3,R32,65.0,W


In [ ]:
# Rename columns to remove "winner_" prefix and "w_" prefix only if they start the column name
player_stats_hist_df.columns = [
    col.replace("winner_", "player_", 1)
    if col.startswith("winner_")
    else col.replace("w_", "p_", 1)
    if col.startswith("w_")
    else col
    for col in player_stats_hist_df.columns
]

player_stats_hist_df_l.columns = [
    col.replace("loser_", "player_", 1)
    if col.startswith("loser_")
    else col.replace("l_", "p_", 1)
    if col.startswith("l_")
    else col
    for col in player_stats_hist_df_l.columns
]

In [ ]:
# Append the loser stats to the winner stats
player_stats_hist_df = pd.concat(
    [player_stats_hist_df, player_stats_hist_df_l], ignore_index=True
)
player_stats_hist_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,player_rank,player_rank_points,p_ace,p_df,p_svpt,p_1stIn,p_1stWon,p_2ndWon,p_SvGms,p_bpSaved,p_bpFaced,score,best_of,round,minutes,results
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,1,101142,1.0,NaN,Emilio Sanchez,R,180.0,ESP,25.6,9.0,1487.0,1.0,0.0,53.0,37.0,30.0,7.0,9.0,5.0,6.0,6-4 6-1,3,R32,63.0,W
1,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,2,101613,NaN,Q,Malivai Washington,R,180.0,USA,21.5,94.0,371.0,5.0,1.0,56.0,25.0,17.0,20.0,9.0,1.0,2.0,6-3 6-2,3,R32,72.0,W
2,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,3,101179,NaN,NaN,Jean Philippe Fleurian,R,185.0,FRA,25.3,77.0,468.0,2.0,4.0,80.0,55.0,35.0,16.0,12.0,2.0,4.0,2-6 6-1 6-2,3,R32,101.0,W
3,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,4,101117,NaN,NaN,Eric Jelen,R,180.0,GER,25.8,65.0,502.0,0.0,1.0,82.0,55.0,35.0,14.0,13.0,6.0,10.0,6-3 1-6 6-4,3,R32,108.0,W
4,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,5,101901,NaN,Q,Chuck Adams,R,185.0,USA,19.7,190.0,142.0,4.0,4.0,65.0,46.0,34.0,12.0,10.0,2.0,2.0,6-3 6-4,3,R32,65.0,W


In [28]:
player_stats_hist_df.shape

(196710, 32)